In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import sys
sys.path.append('/content/gdrive/MyDrive/TFG/notebooks/')


In [3]:
from functions import *

Dependencies installed successfully.


In [4]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import os
import rasterio
import glob
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas as pd
import geopandas as gpd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pyproj
from geopandas.tools import sjoin
from shapely.geometry import Point
import requests
from scipy.stats import ks_2samp
from rasterio.transform import from_origin
import shutil

# Load data

In [5]:
W = 256# with 64 works LR
dir_paths = ['/content/gdrive/MyDrive/TFG/data/raw/lst','/content/gdrive/MyDrive/TFG/data/raw/wt', '/content/gdrive/MyDrive/TFG/data/raw/ndvi']
#dir_paths = ['/content/gdrive/MyDrive/TFG/data/raw/lst','/content/gdrive/MyDrive/TFG/data/external/water temperature interpolated','/content/gdrive/MyDrive/TFG/data/raw/ndvi']
rgb = [True,False,False]
data, time_slots = load_data(dir_paths,W,rgb)
data_inputs = data[dir_paths[0]]
data_targets = data[dir_paths[1]]
ndvi = data[dir_paths[2]]

print('inputs shape:', data_inputs.shape)
print('targets shape:', data_targets.shape)

/usr/local/lib/python3.10/dist-packages/skimage/transform/_warps.py:738: RuntimeWarning: All-NaN slice encountered
  min_val = min_func(input_image)
/usr/local/lib/python3.10/dist-packages/skimage/transform/_warps.py:742: RuntimeWarning: All-NaN slice encountered
  max_val = max_func(input_image)


inputs shape: (125, 256, 256, 3)
targets shape: (125, 256, 256)


In [7]:
start_date = '2013-03'
end_date = '2023-07'

# Generar el rango de fechas en formato YYYY-MM
date_range = pd.date_range(start=start_date, end=end_date, freq='MS').strftime('%Y-%m')

# Convertir a un array de numpy (opcional, si quieres un array en lugar de un pandas Index)
time_slots = date_range.to_numpy()
time_slots

array(['2013-03', '2013-04', '2013-05', '2013-06', '2013-07', '2013-08',
       '2013-09', '2013-10', '2013-11', '2013-12', '2014-01', '2014-02',
       '2014-03', '2014-04', '2014-05', '2014-06', '2014-07', '2014-08',
       '2014-09', '2014-10', '2014-11', '2014-12', '2015-01', '2015-02',
       '2015-03', '2015-04', '2015-05', '2015-06', '2015-07', '2015-08',
       '2015-09', '2015-10', '2015-11', '2015-12', '2016-01', '2016-02',
       '2016-03', '2016-04', '2016-05', '2016-06', '2016-07', '2016-08',
       '2016-09', '2016-10', '2016-11', '2016-12', '2017-01', '2017-02',
       '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08',
       '2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02',
       '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08',
       '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02',
       '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08',
       '2019-09', '2019-10', '2019-11', '2019-12', 

In [6]:
gdf_points = gpd.read_file('/content/gdrive/MyDrive/TFG/data/external/wt_filtered/wt_filtered.geojson')
time_slots = gdf_points.time.dt.strftime("%Y-%m").unique()
months = gdf_points.time.dt.month.unique()

# Delete unuseful data

In [8]:
'''new_inputs = []
new_targets = []
new_masked_inputs = []
del_indices = []
for index in range(0,data_inputs.shape[0]):
  img_gray = np.dot(data_inputs[index][...,:3], [0.2989, 0.5870, 0.1140])

  blacks = img_gray==0
  percentage_b = np.sum(blacks)/blacks.size * 100

  whites = img_gray>=240
  percentage_w = np.sum(whites)/whites.size * 100

  if not np.all(img_gray==255) and not np.all(img_gray==0) and not np.all(img_gray == img_gray[0,0]) and not (percentage_b > 50) and not (percentage_w > 50):
    new_inputs.append(data_inputs[index])
    new_targets.append(data_targets[index])
    new_masked_inputs.append(masked_inputs[index])
  else:
    del_indices.append(index)

new_inputs = np.array(new_inputs)
new_targets = np.array(new_targets)
new_masked_inputs = np.array(new_masked_inputs)
slots = np.delete(time_slots,del_indices)'''

'new_inputs = []\nnew_targets = []\nnew_masked_inputs = []\ndel_indices = []\nfor index in range(0,data_inputs.shape[0]):\n  img_gray = np.dot(data_inputs[index][...,:3], [0.2989, 0.5870, 0.1140])\n\n  blacks = img_gray==0\n  percentage_b = np.sum(blacks)/blacks.size * 100\n\n  whites = img_gray>=240\n  percentage_w = np.sum(whites)/whites.size * 100\n\n  if not np.all(img_gray==255) and not np.all(img_gray==0) and not np.all(img_gray == img_gray[0,0]) and not (percentage_b > 50) and not (percentage_w > 50):\n    new_inputs.append(data_inputs[index])\n    new_targets.append(data_targets[index])\n    new_masked_inputs.append(masked_inputs[index])\n  else:\n    del_indices.append(index)\n\nnew_inputs = np.array(new_inputs)\nnew_targets = np.array(new_targets)\nnew_masked_inputs = np.array(new_masked_inputs)\nslots = np.delete(time_slots,del_indices)'

Including NDVI data

In [9]:
del_indices = []
for index in range(0,data_inputs.shape[0]):
  img_gray = np.dot(data_inputs[index][...,:3], [0.2989, 0.5870, 0.1140])

  blacks = img_gray==0
  percentage_b = np.sum(blacks)/blacks.size * 100

  whites = img_gray>=240
  percentage_w = np.sum(whites)/whites.size * 100

  img_ndvi = ndvi[index]

  whites = np.isnan(img_ndvi)
  percentage_w_ndvi = np.sum(whites)/whites.size * 100

  if np.all(img_gray == 255) or np.all(img_gray == 0) or np.all(img_gray == img_gray[0,0]) or (percentage_b > 50) or (percentage_w > 50) \
    or np.all(np.isnan(img_ndvi)) or np.all(img_ndvi == img_ndvi[0,0]) or (percentage_w_ndvi > 50):
    del_indices.append(index)
  else:
    new_ndvi = np.where(whites, 0.0, img_ndvi) # process ndvi images to not have nans
    ndvi[index]=new_ndvi

In [10]:
slots = np.delete(time_slots,del_indices)

In [11]:
len(slots)

34

# Mask Landsat data

In [12]:
# Selecciona la imagen en el índice 0 (la primera imagen)
river_mask = data_targets[0]  # Cambia el índice según sea necesario

# Imprimir la forma de la imagen seleccionada
print('Selected image shape:', river_mask.shape)

Selected image shape: (256, 256)


In [13]:
# Supongamos que river_mask es tu máscara de río de forma (height, width)
# y que quieres que tenga 3 canales (para las imágenes RGB de data_inputs)
river_mask_expanded = np.expand_dims(river_mask, axis=-1)  # Expande para tener forma (height, width, 1)
river_mask_expanded = np.repeat(river_mask_expanded, data_inputs.shape[-1], axis=-1)  # Expande a (height, width, 3)

# Aplica la máscara solo a las entradas (inputs)
masked_inputs = []

for i,input_img in enumerate(data_inputs):
  if i not in del_indices:
    masked_input = input_img * river_mask_expanded  # Aplica la máscara a los inputs
    masked_inputs.append(masked_input)

# Convierte de nuevo a arrays de numpy
masked_inputs = np.array(masked_inputs)

# Verifica las formas
print('masked inputs shape:', masked_inputs.shape)

masked inputs shape: (34, 256, 256, 3)


# Mask NDVI data

In [14]:
masked_inputs_ndvi = []

for i,input_img in enumerate(ndvi):
  if i not in del_indices:
    masked_input_ndvi = input_img * river_mask  # Aplica la máscara a los inputs
    masked_inputs_ndvi.append(masked_input_ndvi)

# Convierte de nuevo a arrays de numpy
masked_inputs_ndvi = np.array(masked_inputs_ndvi)

# Verifica las formas
print('masked inputs shape:', masked_inputs_ndvi.shape)

masked inputs shape: (34, 256, 256)


# Download preprocessed data

In [15]:
boiron = gpd.read_file('/content/gdrive/MyDrive/TFG/data/external/swissTLMboiron')

bounds = boiron.total_bounds
# Definir la resolución
resolution = 30
x_min, y_min, x_max, y_max = bounds

transform = from_origin(x_min, y_max, resolution, resolution)  # Origen en la esquina superior izquierda

In [ ]:
# Function to clear the contents of a directory
def clear_directory(directory):
    if os.path.exists(directory):
        for file in os.listdir(directory):
            file_path = os.path.join(directory, file)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f'Error deleting {file_path}. Reason: {e}')

In [ ]:
def save_raster(raster_array,filepath,shp):
  resolution = 30
  x_min, y_min, x_max, y_max = shp.total_bounds
  transform = from_origin(x_min, y_max, resolution, resolution)

  with rasterio.open(
        filepath,
        'w',
        driver='GTiff',
        height=raster_array.shape[0],
        width=raster_array.shape[1],
        count=len(raster_array.shape),
        dtype=raster_array.dtype,
        crs=shp.crs.to_string(),  # Ensure correct CRS
        transform=transform,
        nodata=0.0
) as dst:
    if len(raster_array.shape) == 3:
      dst.write(raster_array[:, :, 0], 1)  # Red channel
      dst.write(raster_array[:, :, 1], 2)  # Green channel
      dst.write(raster_array[:, :, 2], 3)  # Blue channel
    else:
      dst.write(raster_array, 1)

Clean preprocessed directories

In [ ]:
raw_data_path = '/content/gdrive/MyDrive/TFG/data/raw/'
preprocessed_data_path = '/content/gdrive/MyDrive/TFG/data/preprocessed/'

data = ['wt', 'lst', 'ndvi', 'wt_interpolated']#, 'lst/masked']
for d in data:
  dest_dir = os.path.join(preprocessed_data_path, d)
  os.makedirs(dest_dir, exist_ok=True)
  clear_directory(dest_dir)
  if d == 'lst':
    dest_dir = os.path.join(dest_dir, '/masked')
    os.makedirs(dest_dir, exist_ok=True)
    clear_directory(dest_dir)

In [ ]:
dest_dir = os.path.join(preprocessed_data_path, 'lst/masked')
os.makedirs(dest_dir, exist_ok=True)
clear_directory(dest_dir)

Copy preprocessed images

In [ ]:
for d in data:
  source_dir = os.path.join(raw_data_path, d)
  dest_dir = os.path.join(preprocessed_data_path, d)

  # List all files in the source directory
  all_images = os.listdir(source_dir)
  count = 0
  # Loop through each file and check if it contains any of the slots
  for ind, image_file in enumerate(all_images):
      # Check if the image filename contains any of the time slots
      if any(slot in image_file for slot in slots):
          # Construct full file paths
          if d == 'ndvi':
            date = image_file.split('_')[1].split('.')[0]
            name = f'{dest_dir}/ndvi_{date}.tiff'
            print(np.sum(np.isnan(ndvi[ind])))
            save_raster(ndvi[ind], name, boiron)
            plt.imshow(ndvi[ind])
            plt.show()
            print(f'Copied: ndvi_{date}, at {dest_dir}')
          else:
            dst_path = os.path.join(dest_dir, image_file)
            src_path = os.path.join(source_dir, image_file)
            # Copy the file to the destination folder
            shutil.copy(src_path, dst_path)
            print()
            print('Image', count)
            print(f'Copied: {image_file}, from {src_path}, to {dst_path}')

            if d == 'lst':
              date = image_file.split('_')[1].split('.')[0]
              name = f'{dest_dir}/masked/lst_masked_{date}.tiff'
              save_raster(masked_inputs[ind], name, boiron)
              print(f'Copied: lst_masked_{date}')

          count+=1




Output hidden; open in https://colab.research.google.com to view.

CHECK

In [ ]:
dir_paths = ['/content/gdrive/MyDrive/TFG/data/preprocessed/ndvi']
W = 256
list_rgb = [False]
data, time_slots = load_data(dir_paths,W,list_rgb)

In [ ]:
n = data[dir_paths[0]]
print(n.shape)
for i in n:
  print(np.sum(np.isnan(i)))
  plt.imshow(i)
  plt.show()

Output hidden; open in https://colab.research.google.com to view.

# Pixel wise dataset creation

In [17]:
river_indices = np.nonzero(river_mask.reshape(-1))[0]

In [18]:
len(river_indices)

4052

### LST dataset

In [ ]:
pixel_ds_lst = pd.DataFrame(columns=[river_indices])
pixel_ds_lst

,29,30,31,32,285,286,287,288,541,542,...,65207,65208,65209,65210,65461,65462,65463,65464,65465,65466


In [ ]:
for i,image in enumerate(masked_inputs):
  img_gray = np.dot(image[...,:3], [0.2989, 0.5870, 0.1140]) # turn to gray image, calculate the weighted average of each channel contribution

  flat_image = img_gray.flatten()

  river_image = flat_image[river_indices]
  pixel_ds_lst.loc[slots[i]] = river_image

In [ ]:
pixel_ds_lst

,29,30,31,32,285,286,287,288,541,542,...,65207,65208,65209,65210,65461,65462,65463,65464,65465,65466
2013-06,44.908076,147.295560,1.181710e+02,48.733964,57.722422,143.451364,61.752930,0.001196,10.965808,150.137229,...,123.831116,23.025738,0.004513,0.004026,6.841812e-04,0.003486,31.504103,154.815837,188.962313,170.367884
2014-01,5.258128,16.870674,1.289863e+01,5.159278,6.312154,14.889073,6.233140,0.000117,1.059019,14.576432,...,26.019301,5.059771,0.001117,0.001167,1.555817e-04,0.000770,6.681184,34.082292,44.741005,46.071424
2014-09,74.244779,243.985153,1.976904e+02,81.261562,95.688379,237.293438,101.414348,0.001972,18.214841,249.732429,...,249.137014,45.192323,0.008348,0.006663,1.424872e-03,0.007102,63.777997,305.946233,357.168090,293.763953
2015-03,0.046515,0.000001,1.103007e-07,0.052682,0.454361,0.837865,0.039577,0.000001,0.106523,1.466190,...,0.000000,0.000000,0.000000,0.000000,2.352714e-12,0.000000,0.000000,0.000000,0.000000,0.000000
2016-07,23.676014,79.626724,6.514830e+01,26.972961,30.833835,77.745303,33.574409,0.000657,5.930505,81.628007,...,69.951017,12.892839,0.002502,0.002166,3.899003e-04,0.001965,17.636404,86.642802,103.931955,88.310065
2016-09,41.064461,134.975501,1.092237e+02,44.794382,52.486633,129.568276,55.757502,0.001092,9.906790,134.581449,...,115.778629,21.873299,0.004384,0.004062,6.364666e-04,0.003234,29.280693,145.940818,181.054811,168.542540
2016-10,2.444782,6.823473,5.582770e+00,1.873959,2.335568,5.553235,2.398173,0.000046,0.317706,4.372935,...,25.115579,4.796207,0.001020,0.001078,1.405114e-04,0.000721,6.435354,31.908634,42.476906,44.463675
2016-12,61.065876,203.799091,1.653247e+02,68.343042,79.757067,198.983726,85.522942,0.001666,15.249870,209.900584,...,198.805765,36.499262,0.007013,0.005809,1.123692e-03,0.005705,52.228221,259.367000,305.611150,251.698838
2017-01,57.321903,190.622021,1.546204e+02,63.579123,73.967412,184.645098,79.205257,0.001549,14.084950,193.866516,...,145.149277,27.299648,0.005442,0.005002,7.985839e-04,0.004071,37.144082,184.769455,227.554481,208.935606
2017-03,47.498184,158.775197,1.298285e+02,53.752123,61.448034,152.331663,65.587899,0.001277,11.600599,157.895190,...,68.917806,12.695657,0.002481,0.002234,3.983959e-04,0.002007,17.956521,87.822987,108.173366,96.633681


In [ ]:
pixel_ds_lst.to_csv('/content/gdrive/MyDrive/TFG/data/preprocessed/lst/leboiron_lst.csv')

### NDVI dataset

In [ ]:
pixel_ds_ndvi = pd.DataFrame(columns=[river_indices])
pixel_ds_ndvi

,29,30,31,32,285,286,287,288,541,542,...,65207,65208,65209,65210,65461,65462,65463,65464,65465,65466


In [ ]:
for i,image in enumerate(masked_inputs_ndvi):
  flat_image = image.flatten()
  river_image = flat_image[river_indices]

  #print(flat_image.shape, river_image.shape)

  pixel_ds_ndvi.loc[slots[i]] = river_image

In [ ]:
pixel_ds_ndvi

,29,30,31,32,285,286,287,288,541,542,...,65207,65208,65209,65210,65461,65462,65463,65464,65465,65466
2013-06,0.708122,2.355711,1.885620,0.866043,0.826926,2.163267,0.988510,0.000020,0.142427,1.988017,...,1.471125,0.327579,0.000069,0.000066,1.044187e-05,0.000053,0.449206,1.800790,1.884345,1.996136
2014-01,0.751625,2.162700,1.669478,0.675885,0.848653,2.082095,0.926581,0.000018,0.159479,2.142450,...,1.432385,0.357089,0.000069,0.000051,9.901169e-06,0.000050,0.385218,1.978192,2.138198,2.160939
2014-09,1.125756,3.779626,3.009267,1.260052,1.465704,3.698269,1.591532,0.000031,0.269666,3.825585,...,1.678288,0.359755,0.000075,0.000068,9.876884e-06,0.000050,0.402234,1.960484,2.135412,2.049745
2015-03,0.240030,0.778066,0.605268,0.214756,0.314019,0.851740,0.337156,0.000005,0.050653,0.915542,...,0.251079,0.050153,0.000011,0.000009,1.774695e-06,0.000007,0.058214,0.302785,0.367870,0.339188
2016-07,0.629656,2.096673,1.746718,0.760279,0.755032,1.993861,0.923639,0.000017,0.128840,1.955254,...,1.251773,0.300313,0.000064,0.000046,8.109668e-06,0.000041,0.331938,1.512280,1.644178,1.369045
2016-09,1.171617,3.898828,3.154933,1.312597,1.524903,3.837851,1.669093,0.000032,0.294727,4.054570,...,1.770694,0.366513,0.000081,0.000067,1.132692e-05,0.000057,0.464658,1.970777,2.092158,2.021664
2016-10,0.575510,1.721317,1.402862,0.539540,0.718798,1.626031,0.750049,0.000013,0.127160,1.662756,...,1.089590,0.280035,0.000052,0.000038,8.211437e-06,0.000040,0.305544,1.473009,1.621369,1.686822
2016-12,1.070429,3.524423,2.907917,1.220738,1.390463,3.497987,1.511767,0.000029,0.265015,3.685870,...,1.597880,0.348775,0.000072,0.000063,1.047528e-05,0.000052,0.439795,1.830613,1.995637,1.887368
2017-01,0.162879,0.554117,0.445217,0.168557,0.213446,0.533986,0.233797,0.000004,0.040431,0.565473,...,0.150257,0.027275,0.000005,0.000005,8.134927e-07,0.000004,0.038036,0.179665,0.223400,0.207807
2017-03,0.539298,1.760293,1.337880,0.531033,0.663736,1.651031,0.730186,0.000013,0.116858,1.763651,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
pixel_ds_ndvi.to_csv('/content/gdrive/MyDrive/TFG/data/preprocessed/ndvi/leboiron_ndvi.csv')

### Water temperature dataset

In [19]:
pixel_ds_wt = pd.DataFrame(columns=[river_indices])
pixel_ds_wt

,29,30,31,32,285,286,287,288,541,542,...,65207,65208,65209,65210,65461,65462,65463,65464,65465,65466


In [21]:
c = 0
for i in range(len(data_targets)):
  if i not in del_indices:
    image = data_targets[i]
    flat_image = image.flatten()
    river_image = flat_image[river_indices]
    pixel_ds_wt.loc[slots[c]] = river_image
    c += 1

In [22]:
pixel_ds_wt

,29,30,31,32,285,286,287,288,541,542,...,65207,65208,65209,65210,65461,65462,65463,65464,65465,65466
2013-06,2.870015,9.654258,7.900476,3.271294,3.759435,9.420562,4.067366,0.000080,0.718058,9.883817,...,6.145782,1.171358,0.000238,0.000225,0.000033,0.000169,1.545363,7.782999,9.784155,9.248206
2014-01,1.629292,5.479551,4.483296,1.856194,2.134468,5.350365,2.310644,0.000045,0.408157,5.617906,...,3.027052,0.577923,0.000118,0.000112,0.000016,0.000083,0.762449,3.846584,4.845422,4.584611
2014-09,3.470196,11.672450,9.551773,3.954965,4.545773,11.391972,4.918897,0.000096,0.868501,11.954502,...,6.936604,1.322857,0.000270,0.000255,0.000037,0.000190,1.745233,8.795513,11.066760,10.467578
2015-03,1.875099,6.305809,5.159029,2.135947,2.456527,6.157527,2.659155,0.000052,0.469722,6.465285,...,3.505339,0.668805,0.000136,0.000129,0.000019,0.000096,0.882348,4.448438,5.599468,5.296111
2016-07,3.718413,12.508859,10.237086,4.238910,4.870588,12.203864,5.268697,0.000103,0.929993,12.801191,...,7.862251,1.499030,0.000305,0.000289,0.000042,0.000215,1.977657,9.964449,12.533132,11.851870
2016-09,3.704787,12.462427,10.198731,4.222960,4.852862,12.160157,5.250077,0.000103,0.926800,12.757146,...,7.566386,1.442626,0.000294,0.000278,0.000040,0.000207,1.903243,9.589507,12.060917,11.404564
2016-10,2.723724,9.160596,7.495629,3.103464,3.568168,8.943588,3.862276,0.000076,0.682141,9.389137,...,5.089096,0.970647,0.000198,0.000187,0.000027,0.000139,1.280566,6.454267,8.120408,7.679015
2016-12,1.547124,5.201037,4.254041,1.761032,2.027250,5.083947,2.196395,0.000043,0.388300,5.344253,...,2.237978,0.427301,0.000087,0.000083,0.000012,0.000061,0.563734,2.843848,3.581138,3.386405
2017-01,0.850973,2.858668,2.336710,0.967038,1.115509,2.800255,1.210750,0.000024,0.214427,2.950813,...,0.744969,0.142829,0.000029,0.000028,0.000004,0.000020,0.188433,0.954024,1.206234,1.141472
2017-03,2.076275,6.983463,5.714106,2.365906,2.719829,6.815886,2.942880,0.000058,0.519632,7.152456,...,4.138843,0.789359,0.000161,0.000152,0.000022,0.000113,1.041394,5.248202,6.602268,6.242376


In [23]:
pixel_ds_wt.to_csv('/content/gdrive/MyDrive/TFG/data/preprocessed/wt_interpolated/leboiron_wt_interpolated.csv')